In [59]:
import matplotlib.pyplot as plt
from graphics import *
from operator import *
from random import *
import numpy as np
import timeit
from math import *

In [60]:
def initialize_agents(speed, N, width, height):
    """
    Initializes our agent set with randomly directed speeds, draws the window and the agents
    """
    agents = [Point(width*random(), height*random()) for i in range(N)]
    speeds = [np.array([0.0, 0.0]) for i in range(N)]
    
    for i in range(N):
        theta = np.pi * random()
        speeds[i][0] = speed * np.cos(theta)
        speeds[i][1] = speed * np.sin(theta)

    return agents, speeds


def initialize_window(agents, width, height):
    win = GraphWin("Swarm", width, height) # size of box
    for agent in agents:
        agent.draw(win)
    win.getMouse()
    return win


def couple_speeds(agents, speeds, a, s, N):
    """
    Simplest model: for each agent, it will give its nearest neighbour a fraction of its speed and re-normalize it
    
    """
    nearest_neighbours = [nearest_neighbour(agent, agents, N) for agent in agents]
    for i in range(N):
        weightedSpeed = a * speeds[nearest_neighbours[i]]
        speeds[i] = speeds[i] + weightedSpeed
        speeds[i] = s * normalized(speeds[i])


def angle(vec1, vec2):  # (Not so) stupid way of doing an angle
    return np.arccos(np.dot(normalized(vec1),normalized(vec2)))
            
def get_distances(agent, agents, N):
    """
    Given one angent and the set of all agents, 
    computes the distances from the first to all of the others
    """
    dists = N * [0.0]
    for i in range(N):
        relative_pos()
        if dists[i] == 0:
            dists[i] = 0.1
    
    return dists

def relative_pos(agent1, agent2):
    dx = agent2.getX() - agent1.getX()
    dy = agent2.getY() - agent1.getY()

    return np.array([dx, dy]), np.linalg.norm([dx, dy])

def nearest_neighbour(agent, agents, N):
    """
    Returns the index for the agent with smallest Eucledian distance to agent in question
    """
    distances = get_distances(agent, agents, N)
    j = next(i for i in range(N) if agents[i] is agent)
    distances[j] = distances[j-1] + 1

    return distances.index(min(distances))


def softened_angle(speed, newspeed, s, maxTheta):
    theta = angle(speed, newspeed)
    if maxTheta**2 > theta**2:
        return s*newspeed
    else:
        return np.dot(rot_matrix(maxTheta), speed)    

def in_sight_range(rel_pos, speed1, angle_range):
    return 360*angle(speed1, rel_pos)/np.pi < angle_range
    

def noisy_vector(noise):
    return noise * np.array([2 * random() - 1, 2 * random() - 1])
    
def biaser(speeds, N, s, i, prop, bias, dev_bias, weight):
    #bias = np.array([0.0,1.0])
    #Ns has to be integer 
    Ns = int(N * prop)
    gbias = np.random.normal(bias, dev_bias, )
    for i in range(Ns):
        tot_dir = normalized(normalized(speeds[i]) + weight * gbias)
        
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir
    #bias = np.dot(tot_dir,np.array([[np.cos(rot_bias*i), 0],[0, np.sin(rot_bias*i)]]))
    return bias

def quality (agents, speeds, N, bias, window, old_cm):
#ACCURACY (DIRECTIONS DISTRIBUTION)
    dev = 0.0
    for i in range(N):
        dev += angle(bias, speeds[i])
    dev_avg = dev/(N*2*(np.pi))
    #print (dev_avg)
    
#ELONGATION: SHAPE OF SWARM
    #can be smarter if we make agents become poses before, more globally in  the code
    poses = [np.array([a.getX(), a.getY()]) for a in agents]
    #Center of Mass
    cm = np.mean(poses,axis=0)
    #Standard Deviation
    std = np.std(poses,axis=0)
    #print (cm, std)

    #Elongation >1 means 
    elong = std[1] / std[0]
    #Drawing it
    #old_cm = Point(cm[0],cm[1])
    #old_cm.draw(window)
    #old_cm.setFill("red")

#GROUP DIRECTION
    #Vector
    group_dir = np.array([cm[0] - old_cm[0], cm[1] - old_cm[1]])                
    #print (group_dir)
    # Norm
    group_dir = np.linalg.norm(group_dir)
    #print (group_dir)
    return cm
    


def rigid_boundary(x_bound, y_bound, agents, speeds, N):
    for i in range(N):
        [dx, dy] = [0, 0]
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            speeds[i][0] = -speeds[i][0]
            dx = x_bound - x         
        
        elif x < 0:
            speeds[i][0] = -speeds[i][0]
            dx = -x
        
        if y > y_bound:
            speeds[i][1] = -speeds[i][1]
            dy = y_bound - y
        
        elif y < 0:
            speeds[i][1] = -speeds[i][1]
            dy = -y

        agents[i].move(dx, dy)
        
        
def periodic_boundary(x_bound, y_bound, agents, speeds, N):  #Changed from rigid boundaries do periodic boundary condition
    [dx, dy] = [0, 0]
    for i in range(N):
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            dx = -x_bound         
        
        elif x < 0:
            dx = x_bound
        
        if y > y_bound:
            dy = -y_bound
        
        elif y < 0:
            dy = y_bound

        agents[i].move(dx, dy)

def next_step(agents, speeds, dt, N):
    dxvec = [dt * speeds[i][0] for i in range(N)]
    dyvec = [dt * speeds[i][1] for i in range(N)]
    for i in range(N):
        agents[i].move(dxvec[i], dyvec[i])
        

def normalized(vector):
    if vector[0] == vector[1] == 0:
        return vector
    return vector / np.linalg.norm(vector)


def rot_matrix(theta, unit = "None"):
    if unit in ["None", "deg"]:
        c, s = np.cos(pi * theta / 180), np.sin(pi * theta / 180)
    elif unit == "rad":
        c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s],[s, c]])

def warn_me_args(N_steps, a, dt, N, width, height, s, rr, ro, ra, noise, prop, weight, bias, dTheta):
    if dt*s < rr:
        print("Warning - step length bigger the repultion radius.")
        

#def get_v_avg(speeds, N):
#    return
#   
#def get_potencials(distances, N):
#    potentials = np.zeros(N, N)
#    for i in range(N):
#        for j in range(i+1,N):
#            potentials[i][j] = potentials[j][i] = morse_pot(distances[i][j])
#    return potentials  
#    
#def morse_pot(r, ca = 0.5, cr = 1, la = 3, lr = 0.5):
#    return cr*np.exp(-r/lr) - ca * np.exp(-r)
#
#def weird_paper(agents, N, dt):
#    distances = [get_distances(agent, agents) for agent in agents]
#    potencials = get_potencials(distances, N)
#    for i in range(N):
#            newSpeed = speed[i] + dt * potentials
#            speeds[i] = speeds[i] + weightedSpeed
#            speeds[i] = s * normalized(speeds[i]) ### PROBLEMA?-------------------------------------------------
#
#    return


In [61]:
#COUZIN MODEL IMPLEMENTED WITH REPULSE, ATRACT AND ORIENT ZONES SEPARATED (1ST PAPER)
def couzin(agents, speeds, N, s, noise, dTheta, rr, ro, ra, sight_range):
    # watch only particles in repulsion zone
    for i in range(N):
        r_dir = np.array([0.0, 0.0])
        o_dir = np.array([0.0, 0.0])
        a_dir = np.array([0.0, 0.0])
        repulsion_flag = False    
        
        for j in range(N):     
            if i == j:
                #Eliminate the i-i interaction
                continue

            rel_pos, distance = relative_pos(agents[i], agents[j])
            
            if in_sight_range(rel_pos, speeds[i], sight_range):
                if distance < rr:
                    rel_pos = normalized(rel_pos)
                    r_dir = r_dir + rel_pos
                    repulsion_flag = True

                elif not repulsion_flag:
                    if distance < ro:
                        o_dir = o_dir + speeds[j]

                    elif distance < ra:
                        rel_pos = normalized(rel_pos)
                        a_dir = a_dir + rel_pos
            
        #Out of for (j), we treat now the resulting direction vector
        if repulsion_flag:
            tot_dir =  normalized(- r_dir)
        else:
            tot_dir = normalized(o_dir) + normalized(a_dir)
            tot_dir = normalized(tot_dir)
            
        tot_dir = normalized(normalized(tot_dir) + noisy_vector(noise))
        
        #avoid pts stoping when not interacting
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = softened_angle(speeds[i], tot_dir, s, dTheta)
    return


#COUZIN MODEL (2) IMPLEMENTED WITH ATRACT AND ORIENT ZONES TOGETHER (2ND PAPER)
def couzin2(agents, speeds, N, s, noise, rr, roa, orient, atract):
    # watch only particles inrepuls
    for i in range(N): ### FIX - Eliminate, in some way the i-i interaction
        curr_agent = agents[i]
        distances = get_distances(curr_agent, agents, N)
        r_dir = np.array([0.0, 0.0])
        o_dir = np.array([0.0, 0.0])
        a_dir = np.array([0.0, 0.0])
        repulsion_flag = False    
        
        for j in range(N):
            if i == j:
                continue
            if distances[j] < rr:
                temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                temp_vec = normalized(temp_vec)
                r_dir = r_dir + temp_vec
                repulsion_flag = True
            elif not repulsion_flag:
                if distances[j] < roa:
                    o_dir = o_dir + speeds[j]
                    temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                    temp_vec = normalized(temp_vec)
                    a_dir = a_dir + temp_vec
        #Out of for (j), we treat now the resulting direction vector
        if repulsion_flag:
            tot_dir =  normalized(-r_dir)
        else:
            o_dir = normalized(o_dir)
            a_dir = normalized(a_dir)
            tot_dir = orient*o_dir + atract*a_dir
            tot_dir = normalized(tot_dir)
            
        tot_dir = normalized(tot_dir + noise * np.array([(-1)+2*random(),(-1)+2*random()]))
        
        #avoid pts stoping when not interacting
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir
    return

    
def vicsek(agents, speeds, N, s, noise, r): # s=speed, noise= letter csi temperature factor, r=radius of interaction
    # consider only particles within 'r' from pt_i, align pt_i with v_avg
    for i in range(N):
        distances = get_distances(agents[i], agents)
        tot_dir = np.array([0.0, 0.0])
        
        for j in range(N):
            if distances[j] < r:
                tot_dir = tot_dir + speeds[j]

        tot_dir = s * normalized(normalized(tot_dir) + noise * np.array([(-1)+2*random(),(-1)+2*random()]))
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = tot_dir
    return

def run(N_steps, N, s, dt, width, height):#N_steps, a, dt, N, width, height, s, rr, ro, ra, noise, prop, weight, bias, dev_bias, dTheta, sight_theta):
    """
    Simulates motion of swarm. Recieves following parameters:

    N_steps  - number of steps to perform
    a -  coupling between neighbouring points
    dt - time step to be used 
    N - number of points to be used
    width & heigth of window
    s - module of speed throughout agents

    """
    
    
    agents, speeds = initialize_agents(s, N, width, height)
    window = initialize_window(agents, width, height)
    old_cm=np.array([0.0,0.0])
    
    for i in range(N_steps):
        
        next_step(agents, speeds, dt, N)
        start = timeit.default_timer()
        ## MODEL
        #couple_speeds(agents, speeds, a, s, N)
        #vicsek(agents, speeds, N, s, 0.1, r = 500)
        #couzin(agents, speeds, N, s, noise, dTheta, rr, ro, ra, sight_theta)
        #couzin2(agents, speeds, N, s, noise,20,200, 1.5,1) #(...)last 3 parameters: roa, orient, atract
        interaction(agents, speeds)
        
        
        bias = biaser(speeds, N, s, i, prop, bias, dev_bias, weight)
        
        #INFORMATION TRANSFER: SHAPE & DIRECTION & ALIGNMENT QUALITY 
        #point_cm.undraw()
        old_cm = quality(agents, speeds, N, bias, window, old_cm)
        
        ## BOUNDARY CONDITIONS
        #rigid_boundary(width, height, agents, speeds, N)
        periodic_boundary(width, height, agents, speeds, N)

        #time.sleep(0.02)
        stop = timeit.default_timer()
        #print stop - start
    window.close()
    return


In [62]:
def menu(N_particles = 50, width = 100, height = 100, model = "None"):
#N_particles = 50
#width = 100
#height = 100
#model = 'None'


    repeat = raw_input("Repeat simulation ([y]/n)?\n")
    while repeat not in ['', 'y', 'n']:
        repeat = raw_input("Please enter valid input ([y]/n)?\n")

    if repeat == 'n':
        if model not in ['None', 'smpl', 'czn', 'vsck', 'czn2']:
            print('Models available:\n'
                    'Simple speed coupling............ smpl\n'
                    'Couzin model.....................  czn\n'
                    'Viscek model..................... vsck\n'
                    'Couzin-2 model................... czn2')
            model = raw_input("Please choose a model.")
            while model not in ['smpl', 'czn', 'vsck', 'czn2']:
                model = raw_input("Please enter a valid model.")

        rep_par = raw_input("Use saved parameters ([y]/n)?\n")
        while rep_par not in ['', 'y', 'n']:
            rep_par = raw_input("Please enter valid input ([y]/n)?\n")

        if rep_par in ['', 'y']:
            [a, s, noise, dTheta, r, rr, ro, ra, roa, sight_theta, orient, atract] = \
                load_parameters(parameters_file)


        else:
            if model in ['None', 'smpl']:
                s = input("Speed: s = ")
                a = input("Coupling: a = ")

            elif model == 'czn':
                s = input("Speed: s = ")
                noise = input("Noise: noise = ")
                dTheta = input("Max angle of turn: dTheta = ")
                rr = input("Repulsion radius: rr = ")
                ro = input("Orientation radius: ro = ")
                ra = input("Atraction radius: ra = ")
                sight_theta = input("Field of view: sight_theta = ")

            elif model == 'cz2':
                s = input("Speed: s = ")
                noise = input("Noise: noise = ")
                dTheta = input("Max angle of turn: dTheta = ")
                rr = input("Repulsion radius: rr = ")
                roa = input("Orientation/atraction radius: roa = ")
                orient = input("Orientation weight: ra = ")
                atract = input("Atract weight: sight_theta = ")

            elif model == 'vsck':
                s = input("Speed: s = ")
                noise = input("Noise: noise = ")
                r = input("Repulsion radius: rr = ")    
    


        if model in ['None', 'smpl']:
            def interaction(agents, speeds):
                return couple_speeds(agents, speeds, a, s, N_particles)

        elif model == 'czn':
            def interaction(agents, speeds):
                return couzin(agents, speeds, N_particles, s, noise, dTheta, rr, ro, ra, sight_theta)

        elif model == 'cz2':
            def interaction(agents, speeds):
                return couzin2(agents, speeds, N_particles, s, noise, rr, roa, orient,atract)

        elif model == 'vsck':
            def interaction(agents, speeds):
                return vicsek(agents, speeds, N_particles, s, noise, r)

    else:
        load_parameters(parameters_file)
        # run things #

    def run(N_steps, N, s, dt, width, height):#N_steps, a, dt, N, width, height, s, rr, ro, ra, noise, prop, weight, bias, dev_bias, dTheta, sight_theta):
        """
        Simulates motion of swarm. Recieves following parameters:

        N_steps  - number of steps to perform
        a -  coupling between neighbouring points
        dt - time step to be used 
        N - number of points to be used
        width & heigth of window
        s - module of speed throughout agents

        """
        prop = 0.4
        weight = 0.8
        bias = np.array([0.0,-1.0])
        dev_bias = 0.1
        rot_bias = 1


        agents, speeds = initialize_agents(s, N, width, height)
        window = initialize_window(agents, width, height)
        old_cm=np.array([0.0,0.0])

        for i in range(N_steps):

            next_step(agents, speeds, dt, N)
            start = timeit.default_timer()
            ## MODEL
            #couple_speeds(agents, speeds, a, s, N)
            #vicsek(agents, speeds, N, s, 0.1, r = 500)
            #couzin(agents, speeds, N, s, noise, dTheta, rr, ro, ra, sight_theta)
            #couzin2(agents, speeds, N, s, noise,20,200, 1.5,1) #(...)last 3 parameters: roa, orient, atract
            interaction(agents, speeds)


            bias = biaser(speeds, N, s, i, prop, bias, dev_bias, weight)

            #INFORMATION TRANSFER: SHAPE & DIRECTION & ALIGNMENT QUALITY 
            #point_cm.undraw()
            old_cm = quality(agents, speeds, N, bias, window, old_cm)

            ## BOUNDARY CONDITIONS
            #rigid_boundary(width, height, agents, speeds, N)
            periodic_boundary(width, height, agents, speeds, N)

            #time.sleep(0.02)
            stop = timeit.default_timer()
            #print stop - start
        window.close()
        return

    run(200,50, 5, 0.1, 100, 100)

        
            


In [63]:
# Field of view is around 340º for pigeons as saind in the source:
# https://www.quora.com/What-does-a-pigeons-field-of-view-look-like
#run(N_steps, a, dt, N, width, height, rr, ro, ra, s, noise, prop, weight, bias, dev_bias, dtheta)
menu(50,100,100,"gae")

#run(N_steps = 300, a = 0.1, dt = 2, N = 4, width = 150, height = 150, rr = 10, ro = 20, ra = 100, s = 5, noise = 0, prop = 0.4, weight = 0.5, bias = np.array([0.0,-1.0]), dev_bias=0, dTheta = 120, sight_theta = 300)

Repeat simulation ([y]/n)?



NameError: global name 'load_parameters' is not defined

In [42]:
run(200,50, 5, 0.1, 100, 100)

TypeError: get_distances() takes exactly 3 arguments (2 given)